In [5]:
def ext_soft_load_np(data, file_split_num ,model, model_vec, n_round = 5):
    

    
    prediction_soft = model.predict(data)
    prediction_vec = model_vec.predict(data)

#     prediction_soft = model.predict(data.X_mel_cqt)
#     prediction_vec = model_vec.predict(data.X_mel_cqt)
#     predicted_classes = np.argmax(prediction_soft, axis=1)
    

    
    f = 0
    softmax_val = []
    test_predict = []

    for j,i in enumerate(file_split_num):
        mean_vec = np.mean(prediction_vec[f:f + i],axis = 0)    
#         mean_soft = np.mean(prediction_soft[f:f + i],axis = 0) 
#         mean_soft = np.mean(prediction[f:f + i],axis = 0)
#         mean_soft = np.mean(np.power(prediction[f:f + i],2),axis = 0)
        
        softmax_val.append(np.round(mean_vec,n_round))
#         test_predict.append(mode(predicted_classes[f:f + i])[0][0])
        
        predicted_classes = np.argmax(np.mean(prediction_soft[f:f + i],axis = 0))
        test_predict.append(predicted_classes)
        f += i
    return np.array(softmax_val), np.array(test_predict)

def ext_soft(data, model, model_vec, n_round = 5):
    

    prediction_soft = model.predict(data.X_reshaped)
    prediction_vec = model_vec.predict(data.X_reshaped)

#     prediction_soft = model.predict(data.X_mel_cqt)
#     prediction_vec = model_vec.predict(data.X_mel_cqt)
#     predicted_classes = np.argmax(prediction_soft, axis=1)
    

    
    f = 0
    softmax_val = []
    test_predict = []
    for j,i in enumerate(data.file_split_num):
        mean_vec = np.mean(prediction_vec[f:f + i],axis = 0)    
#         mean_soft = np.mean(prediction_soft[f:f + i],axis = 0) 
#         mean_soft = np.mean(prediction[f:f + i],axis = 0)
#         mean_soft = np.mean(np.power(prediction[f:f + i],2),axis = 0)
        
        softmax_val.append(np.round(mean_vec,n_round))
#         test_predict.append(mode(predicted_classes[f:f + i])[0][0])
        predicted_classes = np.argmax(np.mean(prediction_soft[f:f + i],axis = 0))
        test_predict.append(predicted_classes)
        f += i
    return np.array(softmax_val), np.array(test_predict)


import time
def make_pdf(data, pa):
    var_type = 'o' * data.shape[1]

    std_feature = np.std(data, axis=0)
    print(data.shape)
    d = data.shape[1]
    feature_length = data.shape[0]
    c = (4 / (d + 2) / feature_length) ** (1 / (d + 4))
    bw = std_feature * c
    print(bw)
#     print(bw.shape)
    # bw = bw.transpose()

    bw = bw * pa
    dens = sm.nonparametric.KDEMultivariate(data=data, var_type=var_type, bw=bw * pa)
    return dens

In [6]:
import import_ipynb
import eval_classes
import importlib
importlib.reload(eval_classes)

from eval_classes import *
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score
from tensorflow.keras.models import load_model


known_volume_path = 'C:/Users/GJ/Desktop/연구실/2022SPCUP/spcup_2022_training_part1'
eval_volume_path = 'C:/Users/GJ/PycharmProjects/2022SPCUP/spcup_2022_eval_part1'


known_path, known_label_path = eval_file_path_list(known_volume_path)
eval_path, eval_label_path = eval_file_path_list(eval_volume_path)


rs = 10

pd_label =  pd.read_csv(known_label_path)
known_labels = pd_label['algorithm']
knwon_train_path, known_test_path, y_train_raw, y_test_raw = train_test_split(np.array(known_path),
                                                                      known_labels, test_size=0.2,
                                                                      stratify = known_labels, random_state=rs)

n_mels = 64


train_data = data(knwon_train_path ,known_label_path,n_mels=n_mels)
eval_data = data(eval_path ,eval_label_path,n_mels=n_mels)

for data in [train_data, eval_data]:
    data.extract_mel()
    data.reshape_data()
    
X_train = train_data.X_reshaped
X_eval = eval_data.X_reshaped









AttributeError: module 'matplotlib' has no attribute 'rcParams'

In [272]:
import pickle
# with open('part1_eval_data_train(train4000).p','wb') as f:
#     pickle.dump(train_data,f)
# with open('part1_eval_data_test(eval9000).p','wb') as f:
#     pickle.dump(eval_data,f)

    
    
with open("./part1_data_pkl/part1_eval_data_train_mel(train4000).p", 'rb') as f:
    train_data = pickle.load(f)
with open("./part1_data_pkl/part1_eval_data_test_mel(eval9000).p", 'rb') as f:
    eval_data = pickle.load(f)
    
print(train_data.X_reshaped.shape)
print(eval_data.X_split.shape)

(25977, 64, 32, 1)
(57083, 64, 32)


In [381]:
train_np_file_num = np.load("./part2_code/part2_data_pkl/part2_cnn_train_data_train(train24000(19200))_file_split_num.npy")
train_np= np.load("./part2_code/part2_data_pkl/part2_cnn_train_data_train(train24000(19200)).npy")

print(train_np.shape)
#np 파일 load 해서 하는 코드
model = load_model('./part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5')
model.summary()

model_vec = load_model('./part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5')


model_vec.pop()
model_vec.compile()
# model_vec.summary()


from scipy.stats import mode
n_round = 5
train_soft, train_pred= ext_soft_load_np(train_np,train_np_file_num, model, model_vec, n_round)
eval_soft, eval_pred= ext_soft(eval_data, model, model_vec, n_round)

for k in range(6):
    print(len(eval_pred[eval_pred == k]))


scaler = RobustScaler()
train_sc = scaler.fit_transform(train_soft)
eval_sc = scaler.fit_transform(eval_soft)

print(train_sc[:3])
print(eval_sc[:3])


pa = 0.03
dens = make_pdf(train_sc, pa)

pdf_tr = dens.pdf(train_sc)
pdf = dens.pdf(eval_sc)

pdf = pdf/max(pdf_tr)


(124002, 64, 32, 1)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 64, 32, 1)         4         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 32, 16)        416       
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 32, 16)        64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 32, 32)        12832     
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 32, 64)        51264     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 

In [383]:
print(train_data.X_reshaped.shape)

(25977, 64, 32, 1)


In [384]:
model = load_model('./part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5')
model.summary()

model_vec = load_model('./part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5')


model_vec.pop()
model_vec.compile()
# model_vec.summary()

from sklearn.preprocessing import RobustScaler
from scipy.stats import mode
n_round = 5
train_soft, train_pred= ext_soft(train_data, model, model_vec, n_round)
eval_soft, eval_pred = ext_soft(eval_data, model, model_vec, n_round)

for k in range(6):
    print(len(eval_pred[eval_pred == k]))


# scaler = StandardScaler()
scaler = RobustScaler()

train_sc = scaler.fit_transform(train_soft)
eval_sc = scaler.fit_transform(eval_soft)

print(train_sc[:3])
print(eval_sc[:3])


pa = 0.03
dens = make_pdf(train_sc, pa)

pdf_tr = dens.pdf(train_sc)
pdf = dens.pdf(eval_sc)

pdf = pdf/max(pdf_tr)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 64, 32, 1)         4         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 32, 16)        416       
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 32, 16)        64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 32, 32)        12832     
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 32, 64)        51264     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 6, 64)         0

In [385]:
np.save('part1_train_sc_robust',train_sc)

In [189]:
# import pandas as pd
# dataframe = pd.DataFrame(train_soft)
# dataframe.to_csv('part1_train_before_softmax.csv')

# dataframe1 = pd.DataFrame(eval_soft)
# dataframe1.to_csv('part1_eval_before_softmax.csv')

# print(train_soft.shape)
# print(eval_soft.shape)

(4000, 6)
(9000, 6)


In [309]:
for k in range(6):
    print(len(eval_pred[eval_pred == k]))
#     print(len(mkde_eval_pred[mkde_eval_pred == k]))    
print('\n')

import copy
mkde_eval_pred = copy.copy(eval_pred)

1496
1522
1525
1505
1804
1148




In [270]:
# # for mt in np.arange(-20,20,0.1):
# #     print(len(eval_mean[eval_mean<mt]), mt)
    
# mkde_eval_pred = copy.copy(eval_pred)    
# for k in range(6):
#     print(len(mkde_eval_pred[mkde_eval_pred == k]))
# #     print(len(mkde_eval_pred[mkde_eval_pred == k]))    
# print('\n')

# mkde_eval_pred[eval_mean>3.1] = 5

# for k in range(6):
#     print(len(mkde_eval_pred[mkde_eval_pred == k]))
# #     print(len(mkde_eval_pred[mkde_eval_pred == k]))    
# print('\n')

1496
1522
1525
1505
1804
1148


1487
1522
1525
1505
405
2556




In [389]:
for k in range(6):
    print(len(eval_pred[eval_pred == k]))

print('\n')
# thresh = 0.5e-06

mkde_eval_pred = copy.copy(eval_pred)
# print(mkde_eval_pred[np.where(pdf<thresh)[0]] )
# mkde_eval_pred[np.where(pdf<thresh)[0]] = 5
# print(mkde_eval_pred[np.where(pdf<thresh)[0]] )



    
#     print(mkde_eval_pred[idx][np.where(pdf[idx]<thresh)[0]])
#     mkde_eval_pred[mkde_eval_pred == i][np.where(pdf[mkde_eval_pred == i]<thresh)[0]] = 5
# #     mkde_eval_pred[idx][np.where(pdf[idx]<thresh)[0]] = np.ones(len(np.where(pdf[idx]<thresh)[0]))*5
# #     print(np.ones(len(np.where(pdf[idx]<thresh)[0]))*5)
#     print(mkde_eval_pred[idx][np.where(pdf[idx]<thresh)[0]])
#     print(np.where(pdf[idx]<thresh)[0])


for i, thresh in enumerate([0.5e-09,0.5e-07,1e-07, 0.5e-07 ,2.5e-06]):
    
    idx_label = np.where(mkde_eval_pred == i)[0]
    idx_thresh = np.where(pdf[idx_label]<thresh)[0]
    idx = idx_label[idx_thresh]
    mkde_eval_pred[idx] = 5
print('eval   :', end = '  ')
for k in [ 1485, 1482, 1504, 1521, 1533, 1475]:    
    print(k, end = '  ')  
print('')
print('mkde x :', end = '  ')
for k in range(6):
    print(len(eval_pred[eval_pred == k]), end = '  ')
print('')
print('mkde o :', end = '  ')
for k in range(6):
    print(len(mkde_eval_pred[mkde_eval_pred == k]),end = '  ')    

1496
1522
1525
1505
1804
1148


eval   :  1485  1482  1504  1521  1533  1475  
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1495  1520  1525  1505  1579  1376  

In [ ]:
eval   :  1485  1482  1504  1521  1533  1475  
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1496  1520  1523  1500  1574  1387  
    
    mkde o / pa : 0.03 / th : [0.5e-09,0.5e-07,0.5e-07, 0 ,1e-06]
label 별 length /// 
eval   :  1485  1482  1504  1521  1533  1475  
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1492  1518  1522  1505  1606  1357  

In [ ]:
label 5 : 1474
(label 4 : 0.1704 == 1533)
(label 3 : 0.169 == 1521)
(labael 2 : 0.1672 == 1504
(label 1 : 0.16471 = 1482
(label 0 : 0.1651 = 1485

In [180]:
# mkde_eval_pred[:] = 4
# print(len(mkde_eval_pred[mkde_eval_pred == 4]))

9000


In [ ]:
plt.plot(pdf)
plt.ylim([np.min(pdf),np.sort(pdf)[-1]])

term = 0
n_un = 0
for thresh in np.arange(0.0000001,0.002,0.0000001):
    print(len(pdf[pdf<thresh]), thresh)
    if term < len(pdf[pdf<thresh]) - n_un:
        term = len(pdf[pdf<thresh]) - n_un
        good_th = thresh
        num_5 = len(pdf[pdf<thresh])
    n_un = len(pdf[pdf<thresh])

print('good_th : ', good_th, 'term : ',term, 'num_5 : ', num_5)

In [376]:
#file_name = "n_round  pa   thresh"
file_name = "./submits_log/answer.txt"
with open(file_name,"w") as file:
    for dirName, subdirList, fileList in os.walk(eval_volume_path):
        for i, filename in enumerate(fileList):
            if i != 9000:
                file.write(str(filename))
                file.write(', ')  
#                 file.write(str(eval_pred[i]))
                file.write(str(mkde_eval_pred[i]))
                
                file.write('\n')  
            else:
                file.close()

##file_name = "n_round  pa   thresh, 5 개수(실제 개수 = 1471개)(4 실제 개수 = 1533개) 
parameter = 7_1_0.091 
ACC: 0.8515714285714285

file_name = n_mels_7_0.8_0.091 _ 1832
ACC: 0.8311428571428572

file_name = n_mels_7_0.8_0.077 _ 1443
0.847

file_name = n_mels_7_0.8_0.077 _ 1443  /  

2022 -03-18

::::file_name = "soft전 + std_scaler ++ n_round(소수점) __ pa __ threshold,__ output 5 개수(eval data 에서 실제 개수 = 1471개)

file_name = soft전 + std_scaler++ 5 __0.03__0.0013 __1446 //mode
ACC: 0.8782857142857143

file_name = soft전 + std_scaler++ 5__0.03__0.0013 __1446 //mean
ACC: 0.8792857142857143

file_name = soft전 + std_scaler++ 5__0.03__0.001 __1288 //mean
ACC: 0.8844285714285715

file_name = soft전 + std_scaler++ 5__0.03__0.0007 __1131//mean 
ACC: 0.8878571428571429

file_name = soft전 + std_scaler++ 5__0.03__0.0003 __889//mean 
ACC: 0.8912857142857142

file_name = soft전 + std_scaler++ 5__0.03__0.0001 __0//mean <- cnn 성능 확인(100% ACC : 0.8366)
ACC: 0.8308571428571428 (file 

file_name = soft전 + std_scaler++ 5__0.03__0.0002 __797//mean 
ACC: 0.893

file_name = soft전 + std_scaler++ 5__0.03__0.0001 __694//mean 
ACC: 0.8875714285714286

file_name = soft전 + std_scaler++ 5__0.03__0.002 __1729//mean 
ACC: 0.8722857142857143

2022 -03- 20
file_name = soft전 + std_scaler++ 5__0.5__0.11 __1467//mean 
ACC: 0.868

file_name = soft전 + std_scaler++ 5__0.5__0.05 __823//mean 
ACC: 0.8831428571428571

file_name = soft전 + std_scaler++ 5__0.1__0.0003 __1465//mean 
ACC: 0.8801428571428571

file_name = soft전 + std_scaler++ 5__0.1__0.0003 __798//mean 
ACC: 0.8928571428571429



2022 -3-23
model = './models_log/64_1024_10_mel_label_6_model2.h5', class 6개로 train , mkde x
ACC: 0.9005714285714286

model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train , mkde x

ACC: 0.9535714285714286
RANDOM: 0.8318401480185249



2022-03-24
model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 
mkde o / pa : 0.03 / th : 0.5e-7
label 별 length ///
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1468  1518  1522  1502  1719  1271  

ACC: 0.9574285714285714
RANDOM: 0.5695099768081119


model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 
mkde o / pa : 0.03 / th : 0.5e-6
label 별 length ///
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1396  1504  1514  1495  1642  1449  

////ACC: 0.9497142857142857


model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 
mkde o / pa : 0.03 / th : [0.5e-07,0.5e-06,0.5e-06,0.5e-06,0.5e-06]
label 별 length /// 
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1468  1504  1514  1495  1642  1377  
////ACC: 0.9575714285714285

model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 
mkde o / pa : 0.03 / th : [0.5e-07,0.5e-06,0.5e-06,0.5e-06,0.5e-05]
label 별 length /// 
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1468  1504  1514  1495  1520  1499  
////ACC: 0.9552857142857143

model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 
mkde o / pa : 0.03 / th : [0.5e-07,0.5e-07,0.5e-07,0.5e-07,0.5e-06]
label 별 length /// 
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1468  1518  1522  1502  1642  1348  
////ACC: 0.9595714285714285

model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 
mkde o / pa : 0.03 / th : [0.5e-07,0.5e-07,0.5e-07,0.5e-07,0.5e-05]
label 별 length /// 
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1468  1518  1522  1502  1520  1470   
////ACC: 0.957

model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 
mkde o / pa : 0.03 / th : [0.5e-07,0.5e-07,0.5e-07,0.5e-07,1e-06]
label 별 length /// 
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1468  1518  1522  1502  1606  1384  

ACC: 0.9595714285714285


2022-03-25

model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 12:04:18
mkde o / pa : 0.03 / th : [0.5e-09,0.5e-07,0.5e-07,0.5e-07,1e-06]
label 별 length /// 
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1492  1518  1522  1502  1606  1360  
ACC: 0.9624285714285714




label 5 : 1474
(label 4 : 0.1704 == 1533)
(label 3 : 0.169 == 1521)
(labael 2 : 0.1672 == 1504
(label 1 : 0.16471 = 1482
(label 0 : 0.1651 = 1485
 
 model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 12:04:18
mkde o / pa : 0.03 / th : [0.5e-09,0.5e-06,0.5e-07,0.5e-07,1e-06]
label 별 length /// 
eval   :  1485  1482  1504  1521  1533  1475  
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1492  1504  1522  1502  1606  1374   
 ACC: 0.9614285714285714

model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 12:04:18
mkde o / pa : 0.03 / th : [0.5e-09,0.5e-07,0.5e-07, 0 ,1e-06]
label 별 length /// 
eval   :  1485  1482  1504  1521  1533  1475  
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1492  1518  1522  1505  1606  1357  
ACC: 0.9627142857142857


model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 12:04:18
mkde o / pa : 0.03 / th : [0.5e-09,0.5e-07,0.5e-07, 0 ,0.85e-06]
label 별 length /// 
eval   :  1485  1482  1504  1521  1533  1475  
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1492  1518  1522  1505  1621  1342  
ACC: 0.9622857142857143


model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 12:04:18
mkde o / pa : 0.03 / th : [0.5e-09,0.5e-07,0.5e-07, 0 ,1e-07]
label 별 length /// 
eval   :  1485  1482  1504  1521  1533  1475  
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1492  1518  1522  1505  1698  1265 
ACC: 0.9614285714285714


model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 12:04:18
mkde o / pa : 0.03 / th : [0.5e-09,0.5e-07,0.5e-07, 0 ,1e-06]
label 별 length /// 
eval   :  1485  1482  1504  1521  1533  1475  
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1492  1518  1522  1505  1606  1357  
ACC: 0.9627142857142857

model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 12:04:18
mkde o / pa : 0.03 / th : [0 , 0 , 0, 0 ,1e-06]
label 별 length /// 
eval   :  1485  1482  1504  1521  1533  1475  
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1496  1522  1525  1505  1606  1346 
ACC: 0.964

model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 12:04:18
mkde o / pa : 0.03 / th : [0 , 0 , 0, 0 ,1e-07]
label 별 length /// 
eval   :  1485  1482  1504  1521  1533  1475  
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1496  1522  1525  1505  1698  1254  
ACC: 0.9627142857142857

model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 12:04:18
mkde o / pa : 0.03 / th : [0 , 0 , 0, 0 ,0.15e-05]
label 별 length /// 
eval   :  1485  1482  1504  1521  1533  1475  
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1496  1522  1525  1505  1588  1364 
ACC: 0.9644285714285714


model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 12:04:18
mkde o / pa : 0.03 / th : [0 , 0 , 0, 0 ,0.3e-05]
label 별 length /// 
eval   :  1485  1482  1504  1521  1533  1475  
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1496  1522  1525  1505  1552  1400  
ACC: 0.9635714285714285


model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 12:04:18
mkde o / pa : 0.03 / th : [0 , 0 , 0, 0 ,0.25e-05]
label 별 length /// 
eval   :  1485  1482  1504  1521  1533  1475  
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1496  1522  1525  1505  1566  1386   
ACC: 0.9634285714285714

2022 -3-28 Standartd Scaler -> Robust Scaler

model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 12:04:18
mkde o / pa : 0.03 / th : [0.5e-09,0.5e-07,0.5e-07, 0 ,1e-06]
label 별 length /// 
eval   :  1485  1482  1504  1521  1533  1475  
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1495  1520  1525  1505  1641  1314    
ACC: 0.9648571428571429

model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 12:04:18
mkde o / pa : 0.03 / th : [0.5e-09,0.5e-07,0.5e-07, 0.5e-07 ,2e-06]
label 별 length /// 
eval   :  1485  1482  1504  1521  1533  1475  
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1495  1520  1525  1505  1595  1360     
ACC: 0.965

model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 12:04:18
mkde o / pa : 0.03 / th : [0.5e-09,0.5e-07,0.5e-07, 0.5e-07 ,1.5e-06]
label 별 length /// 
eval   :  1485  1482  1504  1521  1533  1475  
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1495  1520  1525  1505  1620  1335   
ACC: 0.9642857142857143

model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 12:04:18
mkde o / pa : 0.03 / th : [0.5e-09,4e-06,1e-07, 1e-07 ,2e-06]
label 별 length /// 
eval   :  1485  1482  1504  1521  1533  1475  
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1495  1513  1525  1505  1595  1367
ACC: 0.9645714285714285

model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 12:04:18
mkde o / pa : 0.01 / th : [0.5e-09,2e-06,1e-07, 1e-07 ,2e-06]
label 별 length /// 
eval   :  1485  1482  1504  1521  1533  1475  
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1495  1518  1525  1505  1595  1362  
ACC: 0.9647142857142857

model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 12:04:18
mkde o / pa : 0.03 / th : [0.5e-09,2e-06,1e-07, 1e-07 ,2e-06]
label 별 length /// 
eval   :  1485  1482  1504  1521  1533  1475  
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1495  1518  1525  1505  1595  1362  
ACC: 0.9645714285714285

model = './part2_code/best_model_noise/10_part2_label6_spcup_save_last.h5' class 6개로 train 12:04:18
mkde o / pa : 0.03 / th : [0.5e-09,0.5e-07,1e-07, 0.5e-07 ,2.5e-06]
label 별 length /// 
eval   :  1485  1482  1504  1521  1533  1475  
mkde x :  1496  1522  1525  1505  1804  1148  
mkde o :  1495  1520  1525  1505  1579  1376  
ACC: 0.9644285714285714